In [1]:
import time
import datetime
import torch
import random
import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import scale

In [2]:
# для воспроизводимости эксперимента:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [30]:
timesample = 0.004
start_time = 1.0
end_time = 2.0
n_receivers = 80
n_timesamples = int((end_time - start_time)/timesample)

data = np.empty([1,3,n_receivers,n_timesamples])

In [31]:
n_models = 1100

for i in range(n_models):
    if (i+1) % 100 == 0:
        print(i+1,'/',n_models)
    data_Z = pd.read_csv('csv_models_crack_density_Z\\model_{}.csv'.format(i+1))
    model_gather_Z = np.array(data_Z[data_Z.columns[int(start_time/timesample):int(end_time/timesample)]])
    
    data_X = pd.read_csv('csv_models_crack_density_X\\model_{}.csv'.format(i+1))
    model_gather_X = np.array(data_X[data_X.columns[int(start_time/timesample):int(end_time/timesample)]])
    
    model_zeros = np.zeros(shape=(len(model_gather_X),int(end_time/timesample) - int(start_time/timesample)))
    
    model_gather = np.vstack(([model_gather_Z],[model_gather_X],[model_zeros]))
    model_gather = np.reshape(model_gather,(1,3,n_receivers,n_timesamples))
    
    data = np.vstack((data,model_gather))
    
data = data[1:]

100 / 1100
200 / 1100
300 / 1100
400 / 1100
500 / 1100
600 / 1100
700 / 1100
800 / 1100
900 / 1100
1000 / 1100
1100 / 1100


In [32]:
print(data.shape)
print(data[0])

(1100, 3, 80, 250)
[[[ 8.00617505e-04  1.02273724e-03  1.32891117e-03 ...  7.55271685e-06
    4.02872138e-06  5.54585586e-06]
  [ 8.12600134e-04  1.03973970e-03  1.34946941e-03 ...  1.83841039e-05
    1.61839998e-05  1.07307305e-05]
  [ 8.08924204e-04  1.03607518e-03  1.34817208e-03 ...  2.20412476e-05
    1.66781392e-05  1.50394571e-05]
  ...
  [ 1.56030874e-05  4.26389597e-05  6.87852298e-05 ...  4.40955535e-03
    3.73170828e-03 -1.93199632e-03]
  [-9.79333854e-05 -1.42892852e-04 -2.08559941e-04 ... -7.42730498e-03
   -3.09128850e-03  2.96215597e-03]
  [-3.46126268e-04 -5.30554680e-04 -6.34867465e-04 ...  4.74087521e-03
   -3.35848751e-03 -7.05462322e-03]]

 [[ 6.99237135e-05  8.44761380e-05  9.41588223e-05 ... -3.16790974e-05
   -4.79095761e-05 -4.99428861e-05]
  [ 5.52923593e-05  5.61278430e-05  7.12123438e-05 ...  1.61299686e-05
   -2.66639108e-05 -7.83016876e-05]
  [ 3.67287139e-05  4.03100275e-05  4.04187740e-05 ... -3.74700721e-06
   -1.70057610e-05  2.99470703e-05]
  ...
  [ 

In [33]:
# центрирование и нормализация данных
for i in range(n_models):
    for j in range(2):
        data[i,j] = scale(data[i,j], axis=1)

print(data[0])

[[[ 1.53621181e-02  1.97431569e-02  2.57820613e-02 ... -2.80110109e-04
   -3.49616589e-04 -3.19692976e-04]
  [ 1.61391646e-02  2.07631765e-02  2.70685240e-02 ... -2.91518528e-05
   -7.39406476e-05 -1.84956014e-04]
  [ 1.68832980e-02  2.17099586e-02  2.83416090e-02 ...  1.63065969e-04
    4.91069422e-05  1.42870947e-05]
  ...
  [ 6.10024535e-04  1.78496977e-03  2.92125400e-03 ...  1.91565700e-01
    1.62107313e-01 -8.40302124e-02]
  [-3.48904257e-03 -5.45298574e-03 -8.32149023e-03 ... -3.23654526e-01
   -1.34246363e-01  1.30183398e-01]
  [-1.39971988e-02 -2.20915981e-02 -2.66697937e-02 ...  2.09266781e-01
   -1.46207060e-01 -3.08427176e-01]]

 [[ 1.25165901e-02  1.61046107e-02  1.84919566e-02 ... -1.25344231e-02
   -1.65361819e-02 -1.70375113e-02]
  [ 7.36114401e-03  7.51558306e-03  1.03039511e-02 ...  1.21981047e-04
   -7.78846213e-03 -1.73336986e-02]
  [ 3.97687768e-03  4.52674785e-03  4.54344464e-03 ... -2.23771105e-03
   -4.27344260e-03  2.93563309e-03]
  ...
  [ 1.98903394e-02  1.2

In [34]:
frac_params = pd.read_csv('frac_params_crack_density.csv')
frac_params['aspect_ratio1'] *= 1000
print(frac_params['aspect_ratio1'])
#frac_params = np.array(frac_params[['aspect_ratio1']] * 1000) # умножаем, чтобы числа слишком маленькими не были
frac_params = np.array(frac_params[['e1_n']])

0       0.152476
1       0.244568
2       0.935301
3       0.700633
4       0.574909
          ...   
1095    0.544518
1096    0.556375
1097    0.963857
1098    0.754261
1099    0.989970
Name: aspect_ratio1, Length: 1100, dtype: float64


In [35]:
X_train = torch.Tensor(data[:800])
X_validation = torch.Tensor(data[800:1100])

Y_train = torch.Tensor(frac_params[:800])
Y_validation = torch.Tensor(frac_params[800:1100])

In [87]:
import torch.nn as nn
from torch.hub import load_state_dict_from_url


__all__ = ['AlexNet', 'alexnet']


model_urls = {
    'alexnet': 'https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth',
}


class AlexNet(nn.Module):

    def __init__(self, num_classes=1):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


def alexnet(pretrained=False, progress=True, **kwargs):
    r"""AlexNet model architecture from the
    `"One weird trick..." <https://arxiv.org/abs/1404.5997>`_ paper.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    model = AlexNet(**kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['alexnet'],
                                              progress=progress)
        #state_dict['classifier.6.weight'] = state_dict['classifier.6.weight'][0] # !!!!!!!!!1
        temp = state_dict['classifier.6.weight'][0]
        temp = temp.view(1, temp.size(0))
        state_dict['classifier.6.weight'] = torch.tensor(temp, requires_grad=True)
        
        #state_dict['classifier.6.bias'] = state_dict['classifier.6.bias'][0] # !!!!!!!!!1
        temp = state_dict['classifier.6.bias'][0]
        temp = [temp]
        state_dict['classifier.6.bias'] = torch.tensor(temp, requires_grad=True)
        
        model.load_state_dict(state_dict)
    return model

In [83]:
state_dict = load_state_dict_from_url(model_urls['alexnet'], progress=True)
print(state_dict['classifier.6.weight'],'\n',state_dict['classifier.6.weight'].shape)
state_dict['classifier.6.weight'] = state_dict['classifier.6.weight'][0]
temp = state_dict['classifier.6.weight']
temp = temp.view(1, temp.size(0))
print(torch.tensor(temp, requires_grad=True),'\n',torch.tensor(temp, requires_grad=True).shape)

Parameter containing:
tensor([[ 0.0327, -0.0062, -0.0040,  ...,  0.0160,  0.0456, -0.0158],
        [-0.0281,  0.0393, -0.0035,  ..., -0.0250,  0.0265, -0.0159],
        [-0.0019, -0.0004, -0.0081,  ..., -0.0093,  0.0203, -0.0136],
        ...,
        [-0.0249, -0.0350,  0.0131,  ..., -0.0082,  0.0454, -0.0043],
        [ 0.0252, -0.0026, -0.0109,  ..., -0.0091, -0.0615, -0.0009],
        [-0.0039,  0.0090, -0.0018,  ...,  0.0229,  0.0042,  0.0185]],
       requires_grad=True) 
 torch.Size([1000, 4096])
tensor([[ 0.0327, -0.0062, -0.0040,  ...,  0.0160,  0.0456, -0.0158]],
       requires_grad=True) 
 torch.Size([1, 4096])


c:\program files\python37\lib\site-packages\ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [86]:
state_dict = load_state_dict_from_url(model_urls['alexnet'], progress=True)
print(state_dict['classifier.6.bias'],'\n',state_dict['classifier.6.bias'].shape)
temp = state_dict['classifier.6.bias'][0]
temp = [temp]
state_dict['classifier.6.bias'] = torch.tensor(temp, requires_grad=True)
print(state_dict['classifier.6.bias'],'\n',state_dict['classifier.6.bias'].shape)

Parameter containing:
tensor([ 5.3252e-02,  5.6475e-02,  1.2015e-02,  1.0475e-02,  1.4073e-02,
         2.4921e-02,  4.5943e-02, -1.2418e-02, -5.2491e-02, -1.5580e-02,
        -2.1215e-02, -3.3407e-02,  9.5835e-03,  1.8659e-02,  7.1095e-03,
        -2.5249e-02, -2.9553e-03,  6.2285e-03, -3.0338e-02,  1.7713e-02,
         4.8128e-02,  5.5310e-02,  4.2137e-02, -3.4339e-02,  1.1161e-02,
        -3.7005e-02, -3.7998e-02, -2.2497e-02, -1.3564e-02,  1.1125e-01,
         3.1010e-02,  6.8569e-03, -1.5973e-02, -9.2437e-03,  4.3681e-02,
        -2.6168e-02, -3.0454e-03,  2.6335e-02,  1.0302e-02,  2.9396e-02,
        -1.6149e-02,  3.0833e-02,  4.0436e-02,  6.6803e-02,  2.4527e-02,
         4.6312e-02,  6.1914e-03,  8.0594e-02,  5.9732e-02,  6.1413e-02,
         1.6579e-03,  6.7179e-02, -4.2294e-03, -1.4659e-02, -6.7676e-02,
        -8.3818e-03, -5.8036e-02,  8.1914e-03,  3.9684e-02,  2.8477e-02,
        -1.2424e-01,  3.9262e-02,  9.1787e-03,  6.8728e-02,  4.0663e-02,
        -1.0124e-02,  1.2239e

In [60]:
print(torch.tensor([1.0,2.0,3.0], requires_grad=True))

tensor([1., 2., 3.], requires_grad=True)


In [88]:
device = torch.device('cpu')

#neural_net = FracAnisotropyNet(4,6,2)
neural_net = alexnet(pretrained=True)
neural_net = neural_net.to(device)

loss = torch.nn.MSELoss()
optimizer = torch.optim.Adam(neural_net.parameters(), lr=3.0e-4)

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [89]:
batch_size = 100
N = len(X_train)

test_loss_history = []
train_loss_history = []

X_validation = X_validation.to(device)
Y_validation = Y_validation.to(device)

start_time = datetime.datetime.now()
epochs = 50

for epoch in range(epochs):
    order = np.random.permutation(N)
  
    loss_sum = 0.0
    for batch_init in range(0,N,batch_size):
        optimizer.zero_grad()
        neural_net.train()
    
        X_batch = X_train[order[batch_init:batch_init+batch_size]].to(device)
        y_batch = Y_train[order[batch_init:batch_init+batch_size]].to(device)
    
        pred = neural_net.forward(X_batch)
        #loss_value = weighted_loss(pred,y_batch)
        loss_value = loss(pred,y_batch)
        loss_sum += loss_value
        loss_value.backward()
    
        optimizer.step()
    
    neural_net.eval()
    pred = neural_net.forward(X_validation)
  
    tr_l = loss_sum/(N/batch_size)
    te_l = loss(pred,Y_validation)
    train_loss_history.append(tr_l.data.cpu())
    test_loss_history.append(te_l.data.cpu())
  
    print('epoch #', epoch+1,':\n\tTrain Loss:\t',tr_l.data.cpu().numpy(),'\n\tTest Loss:\t',te_l.data.cpu().numpy())

educ_time = datetime.datetime.now() - start_time
print('Time elapsed:', educ_time)

epoch # 1 :
	Train Loss:	 124.98528 
	Test Loss:	 0.06599144
epoch # 2 :
	Train Loss:	 0.0339041 
	Test Loss:	 0.003511682
epoch # 3 :
	Train Loss:	 0.0029079935 
	Test Loss:	 0.0008652348
epoch # 4 :
	Train Loss:	 0.0012177973 
	Test Loss:	 0.0008331233
epoch # 5 :
	Train Loss:	 0.000989259 
	Test Loss:	 0.0008363974
epoch # 6 :
	Train Loss:	 0.00092391414 
	Test Loss:	 0.0008344212
epoch # 7 :
	Train Loss:	 0.00090225635 
	Test Loss:	 0.00083241786
epoch # 8 :
	Train Loss:	 0.0008847224 
	Test Loss:	 0.0008381263
epoch # 9 :
	Train Loss:	 0.0008993291 
	Test Loss:	 0.00083966524
epoch # 10 :
	Train Loss:	 0.00087488344 
	Test Loss:	 0.00083872606
epoch # 11 :
	Train Loss:	 0.00089320156 
	Test Loss:	 0.00083530106
epoch # 12 :
	Train Loss:	 0.000889927 
	Test Loss:	 0.0008354015
epoch # 13 :
	Train Loss:	 0.0008521693 
	Test Loss:	 0.0008301225
epoch # 14 :
	Train Loss:	 0.0008820505 
	Test Loss:	 0.0008312681
epoch # 15 :
	Train Loss:	 0.00087648694 
	Test Loss:	 0.00083157263
epoch

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
num = 30
def moving_average(a, n=num): # среднее скользящее среднее
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [ ]:
plt.plot(train_loss_history[10:])
plt.plot(test_loss_history[10:])

In [ ]:
avg_loss_train = moving_average(train_loss_history)
avg_loss_test = moving_average(test_loss_history)
plt.plot(avg_loss_train[10:])
plt.plot(avg_loss_test[10:])

In [ ]:
print('Train Loss: ', avg_loss_train[-1])
print('Test Loss:  ', avg_loss_test[-1])

In [ ]:
pred = neural_net.forward(X_validation)
pred = pred.data.cpu().numpy()
y_val = Y_validation.data.cpu().numpy()
print('Predicted\tReal value\tDifference')
sum_dn = 0.0
sum_e1_n = 0.0
sum_e1_t = 0.0
sum_asp = 0.0
sum_asp_real = 0.0
sum_e1_n_real = 0.0
sum_e1_t_real = 0.0
sum_dn_real = 0.0
sum_dn_sq = 0.0
for i in range(300):
    dif = abs(pred[i]-y_val[i])
    print(pred[i],'\t',y_val[i],'\t',dif)
    sum_e1_n += dif[0]
    #sum_e1_t += dif[1]
    #sum_asp += dif[0]
    #sum_asp_real += y_val[i][0]
    sum_e1_n_real += y_val[i][0]
    #sum_e1_t_real += y_val[i][1]
    #sum_dn += dif[0]
    #sum_dn_sq += dif[0]**2
    #sum_dn_real == y_val[i][0]
    
#mean_e1_n_real = sum_e1_n_real/300
#mean_e1_t_real = sum_e1_t_real/300

In [ ]:
print('Mean e1_n difference: ', sum_e1_n/300)
#print('Mean e1_t difference: ', sum_e1_t/300)
#print('Mean aspect ratio difference: ', sum_asp/300)

#print(mean_asp_real)
print('\nRelative e1_n accuracy: ', sum_e1_n/sum_e1_n_real)
#print('Relative e1_t accuracy: ', sum_e1_t/300/mean_e1_t_real)
#print('Relative aspect ratio accuracy: ', sum_asp/sum_asp_real)
#print('Mean dn difference: ', sum_dn/300)
#print('dn MSE: ', math.sqrt(sum_dn_sq/300))
#print('\nRelative dn accuracy: ', sum_dn/sum_dn_real)

In [ ]:
print(Y_validation)

In [ ]:
print(abs((pred-Y_validation).data.cpu().numpy()))